# Applying ZMS REST API on ZODB directly

In [34]:
import ZODB
import os
from Products.Five.browser.tests.pages import SimpleView
from Products.zms import standard
from Products.zms import rest_api
import json

os.getcwd()

'/home/zope/instance/zms5_desy/notebooks'

In [35]:
# Zope/src/Testing/makerequest.py
def makerequest(app, stdout=None, environ=None):
    from io import BytesIO
    from ZPublisher.BaseRequest import RequestContainer
    from ZPublisher.HTTPRequest import HTTPRequest
    from ZPublisher.HTTPResponse import HTTPResponse

    if stdout is None:
        stdout = BytesIO()
    if environ is None:
        environ = {}
    resp = HTTPResponse(stdout=stdout)
    environ.setdefault('SERVER_NAME', 'nohost')
    environ.setdefault('SERVER_PORT', '80')
    environ.setdefault('REQUEST_METHOD', 'GET')
    req = HTTPRequest(BytesIO(), environ, resp)
    req._steps = ['noobject']  # Fake a published object.
    req['ACTUAL_URL'] = req.get('URL')  # Zope 2.7.4

    # Set default skin so that the request is usable for view look-ups.
    from zope.publisher.browser import setDefaultSkin
    setDefaultSkin(req)

    requestcontainer = RequestContainer(REQUEST=req)
    return app.__of__(requestcontainer)

In [36]:
def makeresponse():
    from io import StringIO
    from ZPublisher.HTTPResponse import HTTPResponse
    stdout = StringIO()
    resp = HTTPResponse(stdout=stdout)
    return resp

In [37]:
# Create a ZODB connection to an existing ZODB database file
try:
	db = ZODB.DB(os.path.join(os.getcwd(), 'Data.fs'))
	conn = db.open()
	root = conn.root
	###{'Application': <Application at >}
except:
	db.close()
	print('Error: Database connection had to be closed before reopened.')

In [38]:
# ZMS as 'context' object <Acquisition.ImplicitAcquisitionWrapper>
context = root.Application.myzmsx.content 
# Default REQUEST <Special Object Used to Force Acquisition>
# REQUEST = conn.root()['Application'].REQUEST

In [39]:
# Add REQUEST to context object
context = makerequest(context)
# Add REQUEST vars
context.REQUEST.set('lang','ger')
context.REQUEST.set('path_to_handle','')
# Add RESPONSE
context.REQUEST.set('RESPONSE', makeresponse())
# Add RESPONSE vars
context.REQUEST.RESPONSE.setHeader('test_header','test_value')
# Simple ZMS-API
langs = context.getLangIds()

print(context)
print(langs)
print(context.REQUEST.lang)
print(context.REQUEST.RESPONSE.getHeader('test_header'))

<Products.zms.zms.ZMS object at 0x7fcc2490dc80 oid 0xd in <Connection at 7fcc249a2d30>>
['ger', 'eng']
ger
test_value


In [40]:
lang = context.REQUEST.lang
request = context.REQUEST

print(context.attr('title'))

print( [ id for id in context.objectIds() if id.startswith('e') ] )

print(context.last_modified(context.e97))

print(rest_api.get_meta_data(context.e97))

jsn = rest_api.RestApiController(context,request).list_tree_nodes(context.e97, request)[1]
print(json.dumps(jsn,indent=2))

ZMS - Python-based contentmanagement system for science, technology and medicine
['e5', 'e12', 'e22', 'e34', 'e45', 'e57', 'e88', 'e90', 'e2', 'e97', 'e101', 'e133']
2022-07-04 22:35
{'id': 'e97', 'meta_id': 'ZMSFlexbox', 'uid': 'uid:a31e70dd-301d-4ccd-bf44-c6c6d77a125c', 'getPath': '/myzmsx/content/e97'}
[
  {
    "id": "e98",
    "meta_id": "ZMSTextarea",
    "uid": "uid:e092ab8b-6511-416b-9987-0754f51b5642",
    "getPath": "/myzmsx/content/e97/e98"
  },
  {
    "id": "e99",
    "meta_id": "ZMSTextarea",
    "uid": "uid:61865102-55db-49c8-873d-a913da6d2757",
    "getPath": "/myzmsx/content/e97/e99"
  },
  {
    "id": "e100",
    "meta_id": "ZMSLinkElement",
    "uid": "uid:eaf50517-05d8-421f-95a6-cb277004f284",
    "getPath": "/myzmsx/content/e97/e100"
  }
]


In [41]:
# Finally close ZODB connection
db.close()